# Imports

In [1]:
from flib.preprocess.feature_engineering import cal_features
import sys
import os
import json

# Generate data

In [2]:
pwd = '/home/edvin/Desktop/flib/'
config_path = pwd + 'flib/AMLsim/paramFiles/10K_accts/conf.json'

os.system(f'cd ../flib/AMLsim && python3 scripts/transaction_graph_generator.py "{config_path}"')
os.system(f'cd ../flib/AMLsim && mvn exec:java -Dexec.mainClass=amlsim.AMLSim -Dexec.args="{config_path}"')

with open(config_path, 'r') as f:
    config = json.load(f)
tx_log_path = os.path.join(config['output']['directory'], config['general']['simulation_name'], config['output']['transaction_log'])

print(f'txs log: {tx_log_path}')

INFO:__main__:Random seed: 0
INFO:__main__:Simulation name: 10K_accts
INFO:__main__:Add 14791 base transactions
INFO:__main__:Generated 10000 accounts.
INFO:__main__:Generated 17170 normal models.
INFO:__main__:Normal model counts {'single': 4000, 'fan_out': 551, 'fan_in': 619, 'forward': 4000, 'mutual': 4000, 'periodical': 4000}
INFO:__main__:Exported 10000 accounts to tmp/10K_accts/accounts.csv
INFO:__main__:Exported 15090 transactions to tmp/10K_accts/transactions.csv
INFO:__main__:Output alert member list to: tmp/10K_accts/alert_members.csv
INFO:__main__:Exported 276 members for 35 AML typologies to tmp/10K_accts/alert_members.csv


[INFO] Scanning for projects...
[INFO] 
[INFO] ---------------------------< amlsim:amlsim >----------------------------
[INFO] Building amlsim 1.0.0
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- exec-maven-plugin:3.1.0:java (default-cli) @ amlsim ---
General transaction interval: 7
Base transaction amount: Normal = 1.000000, Suspicious= 150000.000000
Random seed: 0
Simulation name: 10K_accts
Working directory: tmp/10K_accts/


Sep 03, 2024 3:03:28 PM amlsim.AMLSim initSimulatorName
INFO: Simulator Name: 10K_accts
Sep 03, 2024 3:03:28 PM amlsim.AMLSim initSimulatorName
INFO: Simulator Name: 10K_accts
Sep 03, 2024 3:03:28 PM amlsim.AMLSim initSimulatorName
Sep 03, 2024 3:03:28 PM amlsim.AMLSim initSimulatorName
Sep 03, 2024 3:03:28 PM amlsim.AMLSim executeSimulation
INFO: Transaction log file: outputs/10K_accts/tx_log.csv
Sep 03, 2024 3:03:28 PM amlsim.AMLSim executeSimulation
INFO: Transaction log file: outputs/10K_accts/tx_log.csv
Sep 03, 2024 3:03:28 PM amlsim.AMLSim loadAccountFile
INFO: Account CSV header: ACCOUNT_ID,CUSTOMER_ID,INIT_BALANCE,COUNTRY,ACCOUNT_TYPE,IS_SAR,BANK_ID
Sep 03, 2024 3:03:28 PM amlsim.AMLSim loadAccountFile
INFO: Account CSV header: ACCOUNT_ID,CUSTOMER_ID,INIT_BALANCE,COUNTRY,ACCOUNT_TYPE,IS_SAR,BANK_ID
Sep 03, 2024 3:03:29 PM amlsim.AMLSim loadAccountFile
INFO: Number of total accounts: 10000
Sep 03, 2024 3:03:29 PM amlsim.AMLSim loadAccountFile
INFO: Number of total accounts: 1000

Starting AMLSim Running for 30 steps. Current loop:0
****************************** - Finished running 30 steps 

It took: 4.174 seconds to execute the simulation

Simulation name: 10K_accts
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  6.687 s
[INFO] Finished at: 2024-09-03T15:03:34Z
[INFO] ------------------------------------------------------------------------
txs log: outputs/10K_accts/tx_log.csv


In [3]:
dfs = cal_features('../flib/AMLsim/' + tx_log_path, windows=(3, 10), overlap=0.9, include_edges=False)

In [5]:
df = dfs[1]
display(df[0][0])

,account,bank,sums_spending_0_9,means_spending_0_9,medians_spending_0_9,stds_spending_0_9,maxs_spending_0_9,mins_spending_0_9,counts_spending_0_9,sums_spending_9_18,...,stds_18_27,maxs_18_27,mins_18_27,counts_in_18_27,counts_out_18_27,counts_unique_in_18_27,counts_unique_out_18_27,counts_days_in_bank,counts_phone_changes,is_sar
0,3238,bank_d,5558.55,555.855,578.900,101.646646,674.31,338.97,10.0,4719.42,...,463.975713,293.05,-637.96,3.0,1.0,1.0,1.0,27,0,0
1,3927,bank_d,5994.56,599.456,609.830,255.681170,992.79,175.06,10.0,4099.59,...,952.359697,737.48,-609.36,1.0,1.0,1.0,1.0,28,0,0
2,3080,bank_d,5029.89,502.989,530.265,136.664153,673.19,284.42,10.0,3543.46,...,0.000000,280.94,280.94,1.0,0.0,1.0,0.0,19,0,0
3,3885,bank_d,5146.61,514.661,469.990,150.169421,781.41,329.09,10.0,3704.01,...,443.822642,973.90,346.24,2.0,0.0,1.0,0.0,25,0,0
4,3845,bank_d,5205.31,520.531,534.405,204.613208,773.06,64.82,10.0,5170.27,...,137.023152,614.77,420.99,2.0,0.0,1.0,0.0,27,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,3823,bank_d,4950.83,495.083,452.860,197.249510,836.60,174.16,10.0,4908.11,...,0.000000,-715.34,-715.34,0.0,1.0,0.0,1.0,20,0,0
988,3186,bank_d,5017.46,501.746,535.755,201.431903,760.61,143.20,10.0,4501.18,...,0.000000,-762.74,-762.74,0.0,1.0,0.0,1.0,21,0,0
989,3672,bank_d,4158.74,415.874,472.215,220.598448,726.44,48.23,10.0,5418.51,...,138.988706,-496.08,-768.61,0.0,3.0,0.0,1.0,26,0,0
990,3209,bank_d,5211.99,521.199,497.965,242.298954,918.87,151.21,10.0,4769.16,...,0.000000,-854.94,-854.94,0.0,1.0,0.0,1.0,25,0,0
